In [ ]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict

In [ ]:
with open("varia.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    soup = bs(content, "xml")
with open("navneregister.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    registry = bs(content, "xml")

In [ ]:
d = defaultdict(dict)

docDates,regDates,mscontentscount,biblcount = 0,0,0,0
i=1

for item in soup.findAll("TEI"):
    itemDate = "N/A"
    itemType = item['subtype']
    if itemType == "dedication" or itemType == "greeting":
        xmlid = item['xml:id']
        #print(item["xml:id"])
        try:
            bibl = item.find("bibl")
            title = bibl.find("title")
            #print("Title in bibl element")
            biblcount += 1
        except:
            bibl = item.find("msContents")
            title = bibl.find("title")
            #print("Title in msContents element")
            mscontentscount += 1
        for match in title.findAll():
            match.unwrap()
        newTitle = ""
        for x in title.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
            newTitle+=" "+x
        title = newTitle.strip()
        title = re.sub(' +', ' ',title)
        title = re.sub('\n',"",title)
        itemDate = item.find("docDate")
        try:
            xmldate = itemDate['when']
            docDates+=1

        except:
            itemDate = item.find("date")
            xmldate = itemDate['when']
            regDates+=1
        date = itemDate['when']
        datenumbers = xmldate.replace("-","")
        
        if "Vudat" in xmlid:
            persid = xmlid.replace("Vudat","")
        else:
            persid = xmlid[1:]
        persid = persid.replace(datenumbers,"")
        if "-" in persid: # Indicates date range in xmlid.
            persid = persid[3:] # Remove -XX left behind.
        print("\n")
        d[xmlid]['title'] = title
        print(xmlid,title,itemType,date,"pe/orgID:",persid)
        d[xmlid]['type'] = itemType
        d[xmlid]['date'] = date
        d[xmlid]['ID'] = persid
        i+=1
print(i,"docDate elements:",docDates,"\nregular date elements:",regDates,"\nmscontentscount:",mscontentscount,"\nbibl count:",biblcount)

In [ ]:
iP=0
iO=0
found_persons = []
for item in d:
    whoid = d[item]['ID']
    persid = "pe"+whoid
    orgid = "org"+whoid
    try:
        persdiv = registry.find('div', {'xml:id':persid})
        name = persdiv.find('persName')
        for match in name.findAll():
            match.unwrap()
        newname = ""
        for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
            newname+=" "+x
        name = newname.strip()
        name = re.sub(' +', ' ',name)
        print("PERS - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name)
        iP+=1
        found_persons.append(item)
        d[item]['peID'] = "pe"+whoid
        d[item]['clearname'] = name
    except:
        try:
            orgid = registry.find('div', {'xml:id':orgid})
            name = orgid.find('item',{'rend':'name'})
            for match in name.findAll():
                match.unwrap()
            newname = ""
            for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
                newname+=" "+x
            name = newname.strip()
            name = re.sub(' +', ' ',name)
            print("ORG - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name)
            iO+=1
            found_persons.append(item)
            d[item]['orgID'] = "org"+whoid
            d[item]['clearname'] = name
        except:
            pass
        #print(whoid)
print(iP,"personal names\n",iO,"organizations")

In [ ]:
losttitles = []
for item in d:
    if item not in found_persons and d[item]['ID'] != "NN":
        print(item,"not found. Title",d[item]['title'],"ID:",d[item]['ID'])
        losttitles.append(d[item]['title'])

In [ ]:
i=0
iddict = defaultdict(dict)
orgs = []
pers = []
for orgid in registry.findAll('div', {'xml:id':re.compile('org.*')}):
    name = orgid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=" "+x
    name = newname.strip()
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    orgs.append(name)
    iddict[name]["ID"] = orgid['xml:id']
print(i,"organizations")
i=0
for persid in registry.findAll('div', {'xml:id':re.compile('pe.*')}):
    name = persid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # ??? kan ikke bruke title.contents[0]. Legger til i ny streng som workaround.
        newname+=" "+x
    name = newname
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    iddict[name]["ID"] = persid['xml:id']
    pers.append(name)
print(i,"persons")

In [ ]:
iddict

In [ ]:
titlesstillmissing,nodontadd = [],[]
v=0
for x in losttitles:
    z = x.strip()
    #print("Mod 0:",z)
    z = z.replace("Dedikasjon til","")
    z = z.replace("Hilsen til","")
    z = z.replace("Hilsener til","")
    #print("Mod 1:",z)
    if " i " in z:
        a = z.split(" i ")
    elif " på " in z:
        a = z.split(" på ")
    elif " til " in z:
        a = z.split(" til ")
    elif " bakpå " in z:
        a = z.split(" bakpå ")
    else:
        a = [z]
        #print("Neither",x)
    if "Hilsen" in a[0]:
        z = a[1]
    else:
        z = a[0]
    #print(z)
    z = z.strip()
    #print("Mod 2:",z)
    for xx in orgs:
        if z.lower() in xx.lower():
            print("Mod 3O:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
            for document in d:
                #print(x)
                exists = d.get(document, {}).get('title')
                if exists == x:
                    theid = iddict.get(xx)["ID"]
                    d[document]['orgID'] = theid
                    d[document]['clearname'] = xx.strip()
                    print(x,"/",xx.strip(),exists,"updated to ID",theid)
    for xx in pers:
        if z.lower() in xx.lower():
            #print("Mod 3P:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
            for document in d:
                #print(x)
                exists = d.get(document, {}).get('title')
                if exists == x:
                    theid = iddict.get(xx)["ID"]
                    d[document]['peID'] = theid
                    d[document]['clearname'] = xx.strip()
                    print(x,"/",xx.strip(),exists,"updated to ID",theid)
    if z not in nodontadd:
        titlesstillmissing.append(z)
    #print("")
print(v,"new entries")

In [ ]:
i=0
for x in d:
    #print(x)
    peID = d.get(x, {}).get('peID')
    orgID = d.get(x, {}).get('orgID')
    ID = d.get(x, {}).get('ID')
    clearname = d.get(x, {}).get('clearname')
    if peID:
        i+=1
        print("PEID",x,"=",ID,"/",peID,"=",clearname)
        print(i)
    elif orgID:
        i+=1
        print("ORGID",x,"=",ID,"/",orgID,"=",clearname)
        print(i)
#z = d.get(x['orgID'])

In [ ]:
d

In [ ]:
df = pd.DataFrame.from_dict(d).fillna("N/A")
df.to_csv("knownDict.csv", sep=',', encoding='utf-8',index=False)